In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..")
import d2lzh_pytorch.torch_package  as d2l

from d2lzh_pytorch.utils import FlattenLayer
from d2lzh_pytorch.utils import train_ch5
from d2lzh_pytorch.cnn.model.VGG_model import get_VGG_model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(torch.__version__)
print('device: ', device)

/home/jetson/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2.3.0
device:  cuda


In [2]:

if torch.cuda.is_available():
    print('Pytorch版本: \t', torch.__version__)
    print('CUDA版本: \t', torch.version.cuda)
    print('GPU 是否可用: \t', torch.cuda.is_available())
    print('GPU 数量: \t', torch.cuda.device_count())
    print('GPU 型号: \t', torch.cuda.get_device_name())
    print('GPU 的CUDA算力: \t', torch.cuda.get_device_capability())
    print('GPU 的总显存: \t', torch.cuda.get_device_properties(0).total_memory / 1024 / 1024 / 1024, 'GB')
    print('是否支持TensorCore: \t', torch.cuda.get_device_properties(0).major >= 7)
    print('是否支持BF16数字格式: \t', torch.cuda.is_bf16_supported())
    print('当前GPU的显存使用率: \t', torch.cuda.memory_allocated(0) / torch.cuda.get_device_properties(0).total_memory * 100, '%')
else:
    print("warn: 当前服务器GPU不可用")
pass


Pytorch版本: 	 2.3.0
CUDA版本: 	 12.2
GPU 是否可用: 	 True
GPU 数量: 	 1
GPU 型号: 	 Orin
GPU 的CUDA算力: 	 (8, 7)
GPU 的总显存: 	 7.44171142578125 GB
是否支持TensorCore: 	 True
是否支持BF16数字格式: 	 True
当前GPU的显存使用率: 	 0.0 %


In [3]:
# 数据加载

batch_size = 256
#rootPath = r"/mnt/g1t/ai_data/Datasets_on_HHD/FashionMNIST"
#train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, root=rootPath)

# rootPath = r"/mnt/g1t/ai_data/Datasets_on_HHD/FashionMNIST"
rootPath = r"/home/jetson/files/ai_data/FashionMNIST"

train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size, 
                                                    resize=224,
                                                    root =rootPath 
                                                   )

In [4]:
# 加载已训练好的模型
# m_name = "VGG_net_cpu_2023_06_27_11-53-28.pt"  # 7.8M

# model = torch.load(m_name)

VGG = get_VGG_model()
# VGG.load_state_dict(model.state_dict())

In [20]:
m_name = "VGG_net_cpu_2023_06_27_11-53-28.pt"  # 7.8M

VGG = torch.load(m_name)

In [16]:
next(VGG.parameters()).device

device(type='cpu')

In [11]:
def predict_ch3(net, test_iter, n=6):  #@save
    """预测标签（定义见第3章）"""
    
    err = 0
    succ = 0
    
    for X, y in test_iter:
        # 转移到cuda上计算
        #X = X.to(device)
        #y = y.to(device)

        y_hat = net(X) # .cpu()
        
        trues = d2l.get_fashion_mnist_labels(y)
        preds = d2l.get_fashion_mnist_labels(y_hat.argmax(axis=1))  # 计算结果, 预测结果

        
        titles = [true +'\n' + pred for true, pred in zip(trues, preds)]

        X_resized = F.interpolate(X[0: n], size=(28, 28), mode='nearest')
        # print(X_resized.shape)   # 输出：torch.Size([6, 1, 28, 28])
        
        for true, pred in zip(trues, preds):
            
            if true != pred:
                # print("判断错误: 标签是:%s \t 计算为:%s" % (true, pred))
                err += 1
            else:
                succ += 1
                pass

        # d2l.show_images(X_resized.reshape((n, 28, 28)), 1, n, titles=titles[0:n])
        # break
        pass
    
    succ_per = succ / (succ + err)
    print("正确 %s, 错误: %s, 正确率: %s" % (succ, err, succ_per))

In [12]:
test_n = 6


predict_ch3(VGG,
            test_iter,
            n=test_n)



KeyboardInterrupt: 

In [28]:

# 调用封装的包, 计算正确率

d2l.evaluate_accuracy(VGG, test_iter)

0.9163

In [29]:

print("训练数据上的正确率计算: ")
d2l.evaluate_accuracy(VGG, train_iter)



训练数据上的正确率计算: 


0.9371833333333334